# *MODELO FINAL*

## 1. Selección del Mejor Modelo
Tras evaluar distintos modelos (KNN, Árboles de Decisión, Regresión Logística, SVM, etc) se comprobó que el SVM con Kernel RBF y los hiperparámetros óptimos:
* C=10
* gamma = 'scale'
* kernel = 'rbf'

presenta el mejor desempeño en términos de Balanced Accuracy en el conjunto de validación interna. Esto lo convierte en nuestro candidato para el modelo final

## 2. Evaluación Outer y Estimación del Desempeño futuro
La estrategia de evaluación outer consiste en separar los datos en dos partes:

* Train (2/3): Para realizar el ajuste y selección de modelos
* Test (1/3): Para obtener una estimación real de rendimiento que tendría el modelo en una competición

Aunque durante la práctica se ha usado esta partición para evaluar el desempeño, en el paso final se reentrena al modelo usando todos los datos disponibles. 

In [3]:
import joblib
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder # Añade los imports necesarios si ejecutas esto por separado

# Carga el modelo
modelo_final = joblib.load('modelo_final.pkl')
print("Modelo cargado.")

# Accede al pipeline (asumiendo que es un best_estimator_ de GridSearch/RandomizedSearch)
# Si tu modelo guardado ES el pipeline directamente, quita .best_estimator_
try:
    if hasattr(modelo_final, 'best_estimator_'):
        pipeline_real = modelo_final.best_estimator_
        print("Accediendo a través de best_estimator_")
    else:
        pipeline_real = modelo_final # Asume que el objeto guardado es el pipeline
        print("Accediendo directamente al objeto cargado.")

    # Verifica si es un pipeline y muestra los pasos
    if hasattr(pipeline_real, 'named_steps'):
        print("\nPasos encontrados en el pipeline:")
        print(pipeline_real.named_steps)
        print("\nNombres de los pasos disponibles (claves):")
        print(list(pipeline_real.named_steps.keys()))
    else:
        print(f"\nEl objeto ({type(pipeline_real)}) no parece ser un Pipeline de scikit-learn (no tiene 'named_steps').")

except Exception as e:
    print(f"\nOcurrió un error al inspeccionar el modelo: {e}")
    print("Asegúrate de que 'modelo_final.pkl' contiene el objeto esperado (posiblemente un Pipeline o resultado de GridSearchCV).")

# ---- Aquí continuaría tu código original ----
# ... (carga de datos_test, preprocesamiento, etc.)

Modelo cargado.
Accediendo a través de best_estimator_

Pasos encontrados en el pipeline:
{'imputer': SimpleImputer(), 'scaler': RobustScaler(), 'svm': SVC(C=10)}

Nombres de los pasos disponibles (claves):
['imputer', 'scaler', 'svm']


/home/alba/Documentos/INGENIERÍA INFORMATICA/3er CURSO/2ndo CUATRI/aprendizaje automático/P1_AA/.venv/lib/python3.12/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator SimpleImputer from version 1.4.1.post1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/alba/Documentos/INGENIERÍA INFORMATICA/3er CURSO/2ndo CUATRI/aprendizaje automático/P1_AA/.venv/lib/python3.12/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RobustScaler from version 1.4.1.post1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/alba/Documentos/I

## 4.Generación de Predicciones para la Competición
El siguiente paso es utilizar el modelo final para generar predicciones sobre el conjunto de datos de la competición. Para ello se debe aplicar el mismo preprocesado que a los datos de entrenamiento. El fichero resultante se guardará como predicciones.csv

In [13]:
import pandas as pd
import joblib
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

# Importar los datos de entrenamiento
datos_generales_originales = pd.read_csv('./attrition_availabledata_10.csv.gz')
print("Datos originales de entrenamiento cargados para ajustar el preprocesador.")


# Para el conjunto de competición, eliminamos las mismas columnas que eliminamos en el entrenamiento
datos_generales_originales['Attrition'] = datos_generales_originales['Attrition'].map({'Yes': 1, 'No': 0})
X_entrenamiento = datos_generales_originales.drop(columns=['Attrition', 'EmployeeID', 'Over18', 'EmployeeCount', 'StandardHours'])

# Redefinir y AJUSTAR el ColumnTransformer 
categorical_features = ['Department', 'JobRole', 'EducationField']
ordinal_features = ['BusinessTravel', 'Gender', 'MaritalStatus']
# numerical_features = X_entrenamiento.select_dtypes(include=['int64','float64']).columns.tolist() # No se usa explícitamente aquí
valores_ordinales = [
    ['Non-Travel', 'Travel_Rarely', 'Travel_Frequently'],   # BusinessTravel
    ['Male', 'Female'],                                     # Gender
    ['Single', 'Married', 'Divorced']                       # MaritalStatus 
]

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features), # Añadido handle_unknown por seguridad
        ('ord', OrdinalEncoder(categories=valores_ordinales, handle_unknown='use_encoded_value', unknown_value=-1), ordinal_features) # Añadido manejo de desconocidos
    ],
    remainder='passthrough' 
)

# Ajustamos el preprocesador con TODOS los datos de X 
preprocessor.fit(X_entrenamiento)

# --- PASO 2: Cargar datos de competición y modelo final ---
datos_test_competicion = pd.read_csv('./attrition_competition_10.csv.gz')
modelo_final = joblib.load('modelo_final.pkl') # Warnings de versión aparecerán aquí


# reparar datos de competición 
columnas_a_eliminar_test = ["EmployeeID", "EmployeeCount", "Over18", "StandardHours"]
# Asegúrate de que datos_test_competicion exista y sea el DataFrame correcto
datos_test_competicion_limpios = datos_test_competicion.drop(columns=columnas_a_eliminar_test)
# No eliminamos 'Attrition' aquí porque no existe en los datos de competición

# Aplicar el preprocesador ajustado a los datos de competición
datos_test_transformados = preprocessor.transform(datos_test_competicion_limpios)

# Convertir a DataFrame para facilitar la manipulación
columnas_prefijadas = preprocessor.get_feature_names_out()
datos_test_listos_para_predecir = pd.DataFrame(datos_test_transformados, columns=columnas_prefijadas)

# --- PASO 3: Generar Predicciones ---
print("\nGenerando predicciones...")

# Pasamos el DataFrame con los nombres de columna CORRECTOS (prefijados)
y_pred_test = modelo_final.predict(datos_test_listos_para_predecir)

# Creamos un DataFrame con las predicciones y el EmployeeID original
df_predicciones = pd.DataFrame({
    'EmployeeID': datos_test_competicion['EmployeeID'], # Usamos el ID original
    'Attrition': y_pred_test # La columna de predicción
})

# Mapeamos de nuevo a 'Yes'/'No' si es necesario para el formato de salida
df_predicciones['Attrition'] = df_predicciones['Attrition'].map({1: 'Yes', 0: 'No'})

print(f"\nDistribución de predicciones:\n{df_predicciones['Attrition'].value_counts()}")

# Guardar las predicciones
output_filename = "predicciones.csv"
df_predicciones.to_csv(output_filename, index=False)
print(f"\nPredicciones guardadas en '{output_filename}'")


Datos originales de entrenamiento cargados para ajustar el preprocesador.

Generando predicciones...

Distribución de predicciones:
Attrition
No     1236
Yes     234
Name: count, dtype: int64

Predicciones guardadas en 'predicciones.csv'


/home/alba/Documentos/INGENIERÍA INFORMATICA/3er CURSO/2ndo CUATRI/aprendizaje automático/P1_AA/.venv/lib/python3.12/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator SimpleImputer from version 1.4.1.post1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/alba/Documentos/INGENIERÍA INFORMATICA/3er CURSO/2ndo CUATRI/aprendizaje automático/P1_AA/.venv/lib/python3.12/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RobustScaler from version 1.4.1.post1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/alba/Documentos/I

# 5. Tarea Adicional


Para esta tarea adicional, hemos decicido utilizar la herramienta SHAP, para interpretar nuestro modelo final.

SHAP (SHapley Additive exPlanations) es una herramienta basada en la teoría de juegos que permite cuantificar la contribución de cada variable a la predicción de un modelo. Entre sus principales ventajas destacan:

* **Interpretabilidad Local y Global**: SHAP ofrece explicaciones tanto a nuvel individual (por cada predicción) como a nivel global (importancia de las variables a lo largo de todo el conjunto). Esto permite identificar cómo cada característica influye en cada decisión del modelo.

* **Consistencia y solidez**: Los valores de Shapley garantizan que la contribución de cada característica se mida de forma justa, lo que otorga una interpretación coherente y consistente.

* **Modelo-agnóstico**: A diferencia de otros métodos que se basan en la estructura interna del modelo (por ejemplo, la importancia basada en la impureza de los árboles en RandomForest), SHAP puede aplicarse a cualquier modelo.

* **Transparencia**: Integrar SHAP en el análisis del modelo permite detectar posibles sesgos o relaciones inesperadas entre variables, lo cual es fundamental para generar confianza en la toma de decisiones basadas en inteligencia artificial.

Realizar esta tarea extra con SHAP resulta especialmente interesante en comparación con construir otro modelo como un RandomForest o una red neuronal. Aunque estos modelos pueden ofrecer buenos resultados, la interpretación de sus resultados (por ejemplo, mediante la importancia de variables basada en el Gini o técnicas de regularización) suele ser menos precisa y profunda. En cambio, al interpretar el modelo SVM seleccionado con SHAP, se obtiene una explicación detallada y cuantitativa de la influencia de cada característica en la predicción, lo cual añade un valor extra al análisis, especialmente en contextos donde la explicabilidad es crucial.

## 5.1 Código para usar SHAP en nuestro modelo final

In [ ]:
import shap
import matplotlib.pyplot as plt
import pandas as pd

# Seleccionar una muestra representativa del conjunto preprocesado para el fondo (background)
# Reducir el tamaño del fondo para acelerar el cálculo
X_background = X_preprocessed.sample(20, random_state=42)

# Definir la función de predicción asegurándonos de que se pasan los nombres de las columnas
def model_predict(data):
    # Convertir a DataFrame si es necesario
    if not isinstance(data, pd.DataFrame):
        data = pd.DataFrame(data, columns=columnas_preprocesadas)
    return final_pipeline.predict_proba(data)[:, 1]

# Crear el explicador con KernelExplainer usando el fondo reducido
explainer = shap.KernelExplainer(model_predict, X_background)

# Seleccionar un subconjunto reducido de datos para explicar
X_explain = X_preprocessed.sample(10, random_state=42)

# Calcular los valores SHAP para el subconjunto seleccionado, limitando nsamples para acelerar el proceso
shap_values = explainer.shap_values(X_explain, nsamples=50)

# Visualizar el resumen de los valores SHAP para identificar la importancia de las variables
shap.summary_plot(shap_values, X_explain, feature_names=columnas_preprocesadas)
plt.show()

# Explicación local de una instancia en particular
i = 0  # índice de la instancia a explicar
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[i], X_explain.iloc[i], feature_names=columnas_preprocesadas)



ModuleNotFoundError: No module named 'shap'

La gráfica SHAP permite ver qué variables han influido más en las predicciones del modelo SVM sobre la variable Attrition. También muestra cómo valores altos o bajos de cada variable afectan individualmente a cada empleado.

En este caso, la variable más influyente es remainder__TotalWorkingYears, lo que indica que el número total de años trabajados es clave en la decisión. Empleados con más años suelen tener valores SHAP negativos, lo que reduce la probabilidad de abandono. También destacan variables como remainder__hrs, DistanceFromHome o TrainingTimesLastYear. Por ejemplo, empleados que viven lejos tienden a tener más probabilidad de dejar la empresa, mientras que recibir más formación puede asociarse a menor riesgo.

Las variables categóricas como el estado civil o el rol profesional (JobRole_Manager, MaritalStatus) también influyen, aunque en menor medida. Tener un cargo alto o estar casado parece relacionarse con mayor estabilidad.

En general, el gráfico muestra que la experiencia, la distancia al trabajo y otros factores numéricos son los que más afectan al resultado. Se observan patrones coherentes: valores altos en algunas variables empujan la predicción hacia Attrition = 1, mientras que en otras hacen lo contrario. Esto permite confiar más en el modelo y entender mejor sus decisiones.
